In [21]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 as cv
import pandas as pd
import pygame

pygame 2.1.2 (SDL 2.0.18, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Capítulo 5

In [175]:
P = np.array( [ [0.2, 0.2], [0.7, 0.1] ])
D = np.array( [ [1.1, 0], [0, 0.9]])
Pi = np.linalg.inv(P)

A = P @ D @ Pi
print(A)


[[ 0.86666667  0.06666667]
 [-0.11666667  1.13333333]]




Para todos os auto-vetores:
* $P$ = matriz de auto-vetores (um auto-vetor por coluna)
* $D$ = matriz de auto-valores (na diagonal principal)
$ A P = P D \rightarrow A = P D P^{-1}$


In [156]:
A = np.array( [ [0.8, 0.1] , [0.3, 0.95] ])
print(np.linalg.eig(A))

(array([0.68625414, 1.06374586]), array([[-0.66026926, -0.3545255 ],
       [ 0.75102896, -0.93504634]]))


# Parte 1

# Exercício 1
**Objetivo: Executar um sistema dinâmico e procurar por pontos de estabilidade**


No [capítulo 2](02-sistemas_lineares.ipynb), usamos uma matriz para representar dois sistemas que operam dinamicamente: a população de carcarás e sapos numa região fictícia, e a dinâmica de navegação de um web-surfista aleatório. Se precisar, revise esse material!

Esses dois sistemas são chamados de *sistemas dinâmicos* porque eles estão tratando de uma evolução de elementos ao longo do tempo. Como já falamos de carcarás e navegadores, vamos tratar agora de um outro sistema bem interessante, que é o de bicicletas públicas de Montreal.

Já por perto de 2010 a 2011, Montreal tinha um sistema de bicicletas públicas bastante parecido com as bicicletas Itaú que temos hoje em São Paulo. Só que Montreal é uma cidade que fica em uma montanha (cujo ponto mais alto é o [Mount Royal](https://www.google.com/maps/place/Mount+Royal+Park/@45.5023053,-73.6098349,14.25z/data=!4m13!1m7!3m6!1s0x4cc91a541c64b70d:0x654e3138211fefef!2sMontreal,+QC,+Canada!3b1!8m2!3d45.5018869!4d-73.5673919!3m4!1s0x4cc91a3b89d50ee1:0x4c8dc463a4718c9a!8m2!3d45.5015664!4d-73.5932179)), o que levou a um problema interessante: as pessoas tendem a tomar uma bicicleta emprestada na estação que fica na região mais alta e devolver em uma das estações das regiões mais baixas, mas raramente alguém faz o caminho inverso. O fim dessa história é que a [prefeitura contratou caminhões para ficarem levando bicicletas morro acima](https://www.csmonitor.com/World/Global-News/2011/0921/Montreal-s-public-bike-system-faces-uphill-battle), e uma consequência inesperada é que essa história seria um exemplo numa aula de álgebra linear dez anos depois num outro hemisfério.

---

Em nosso modelo, vamos fazer várias suposições que cabem para o nosso exemplo, mas que não necessariamente correspondem a dados reais.

* Vamos supor que temos três estações. As duas primeiras (E1 e E2) ficam na parte baixa da cidade. A outra estação (E3) fica na parte alta.
* Quando uma bicicleta é tomada emprestada em E1, ela tem 90% de chance de ser devolvida em E2 e 10% de chance de ser devolvida em E3.
* Quando uma bicicleta é tomada emprestada em E2, ela tem 95% de chance de ser devolvida em E1 e 5% de chance de ser devolvida em E3.
* Quando uma bicicleta é tomada emprestada em E3, ela tem 70% de chance de ser devolvida em E1 e 30% de chance de ser devolvida em E2.

Vamos supor que cada estação começa com 10 bicicletas, e que não há um limite máximo de bicicletas que podem ficar em cada estação.

Gostaríamos de saber:

(a) Após alguns dias de iterações aleatórias, quantas bicicletas esperamos encontrar em cada uma das estações?

(b) Com esses dados inventados, vai ser necessário contratar caminhões para levar bicicletas de alguma estação para outra? Para quais?

(c) Se, ao invés de começarmos nossas iterações com 10 bicicletas em cada estação, começarmos com todas as 30 bicicletas na estação E3, no topo do morro, devemos encontrar uma quantidade final de bicicletas diferente, ao fim de várias iterações?

---

Discussão sobre a solução:

Sob um ponto de vista matemático, um ciclista aleatório parece se comportar de uma maneira muito parecida com um web-surfista aleatório! Veja o exercício 14 do capítulo de sistemas lineares para uma explicação sobre isso. Podemos representar nosso vetor de bicicletas como:

$$
x_0 = 30 \begin{bmatrix} 1/3 \\ 1/3 \\ 1/3 \end{bmatrix}
$$

e a matriz que representa as transições entre estações será a matriz $A$ de forma que $a[i,j]=P(s_t = i | s_{t-1}=j)$, isto é, a $a_{i,j}$ é a probabilidade de uma bicicleta ser deixada na estação $i$ sabendo que ela estava estação $j$:

$$
A = \begin{bmatrix} 
    0 & 0.9 & 0.7 \\
    0.95 & 0 & 0.3 \\
    0.05 & 0.1 & 0 
    \end{bmatrix}
$$



In [157]:
# Comecei com as matrizes já digitadas no código 
x = np.array([[1], [1], [1]])/3
A = np.array( [ [0, 0.95, 0.05], [0.9, 0, 0.1], [0.7, 0.3, 0]]).T
# um passo: x = A @ x
# implemente 100 passos seguidos!
for _ in range(100):
    x = A@x

# Número de bicicletas por estação = # total de bicicletas * P(bicicleta por estação)
print(30*x)

[[13.99017662]
 [13.91849002]
 [ 2.09133335]]


# Exercício 2
**Objetivo: Entender auto-vetores e auto-valores como pontos de estabilidade, expansão, ou colapso**

O problema das bicicletas de Montreal (assim como o PageRank e o problema dos carcarás do Capítulo 2) está ligado a uma característica de matrizes que é a existência de vetores e valores chamados de auto-valores e de auto-vetores. Quando um auto-vetor de uma matriz é multiplicado pela própria matriz, o resultado é um múltiplo do próprio vetor, isto é:

$$
Ax = x \lambda,
$$
onde: $A$ é a matriz, $x$ é o auto-vetor e $\lambda$ é o auto-valor correspondente.

Se tivermos algum vetor na mesma direção de $x$ mas multiplicado por algum número real, isto é, $y=\alpha x$, podemos manter a mesma equação:

$$
A \alpha x = \alpha x \lambda \rightarrow Ay = y \lambda.
$$

Essa equação é incrivelmente importante e vamos retomá-la ao longo deste capítulo. Por enquanto, vamos pensar no problema da população de carcarás e de sapos. Nesse problema, tínhamos a situação em que carcarás estão no mesmo habitat que sapos, obedecendo às seguintes regras:

* A população de carcarás naturalmente cai em 20% a cada mês
* Para cada cinco mil sapos existentes naquele mês, um novo carcará nasce
* A população de sapos naturalmente sobe em 10% porque eles se reproduzem
* A cada mês, cada carcará consegue comer 100 sapos

Isso significa que podemos escrever equações para calcular o número de carcarás e de sapos a cada mês, tomando por base a quantidade deles no mês anterior:

$$ 
\begin{cases}
    \begin{aligned}
    c_t & = 0.8 c_{t-1} + 0.2 s_{t-1} \\
    s_t & = - 0.1 c_{t-1} + 1.1 s_{t-1} \\
    \end{aligned}
\end{cases}
$$

e, portanto, podemos fazer esse cálculo usando a operação matricial:
$$
\begin{bmatrix}
    c_t \\
    s_t 
\end{bmatrix}
=
\begin{bmatrix}
    0.8 & 0.2\\
    -0.1 & 1.1
\end{bmatrix} 
\begin{bmatrix}
    c_{t-1} \\
    s_{t-1}
\end{bmatrix}
$$

Vamos supor duas condições iniciais: 500 carcarás e 500 mil sapos, e, após, 800 carcarás e 400 mil sapos. O que acontece ao longo do tempo com nosso ecossistema com cada uma dessas condições?

In [177]:
A = np.array([[0.8, 0.2], [-0.1,  1.1]])
#x = np.array([[500],[500]])
x = np.array([[800], [400]])
for _ in range(100):
    x = A @ x

print(x)

[[0.02124912]
 [0.01062456]]


# Exercício 3
*Objetivo: calcular auto-vetores e auto-valores de uma matriz usando o pacote Python*

Uma matriz quadrada $N \times N$ tem $N$ autovetores de tamanho $N$, e $N$ autovalores correspondentes. Eles podem ser calculados usando a função `np.linalg.eig(A)`, onde `A` é a matriz da qual querremos calcular os auto-valores e auto-vetores.

A função `np.linalg.eig()` retorna duas estruturas `np.array`. A primeira é um vetor contendo autovalores. A segunda é uma matriz contendo um autovetor por coluna.

(a) Usando a função `np.linalg.eig`, encontre os autovalores e autovetores da matriz $A$ definida no código abaixo:

In [195]:
A = np.array( [[1, 2, 3, 4], [4, 3, 2, 0], [0, 0, 0, 1], [1, 0, 2, 3]])

(b) Usando a função `np.linalg.eig`, encontre os autovalores e autovetores da matriz A correpondente ao sistema de carcarás e sapos acima.

In [ ]:
A = np.array([[0.8, 0.2], [-0.1,  1.1]])


(c) Relacione os autovalores e autovetores que você encontrou no ítem (b) com o comportamento que foi observado no exercício anterior: como podemos saber que a inicialização $(500,500)$ leva à estabilidade, e a inicialização $(800,400)$ leva ao colapso?

# Exercício 4
*Objetivo: encontrar uma matriz à partir de seus auto-valores e auto-vetores*

Em algumas situações (por exemplo, quando estou montando essa lista de exercícios - mas, de forma mais geral, quando estamos projetando sistemas dinâmicos), gostaríamos de encontrar matrizes que têm os autovalores e autovetores que queremos. Para isso, vamos escrever a equação de autovetores e autovalores na forma matricial.

Quando estamos falando de somente um auto-vetor, podemos escrever:

$$
Ax = x \lambda
$$

Porém, se tivermos dois auto-vetores, e seus auto-valores correspondentes, temos na verdade um sistema:

$$
\begin{cases}
Ax_1 = x_1 \lambda_1 \\
Ax_2 = x_2 \lambda_2 
\end{cases}
$$

Esse sistema pode ser escrito na forma de uma multiplicação matricial, se assumirmos que nossos auto-vetores são vetores-coluna:

$$
A \begin{bmatrix} x_1 & x_2 \end{bmatrix} = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} 
$$

Multiplicando os dois lados da equação por $\begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}$, ficamos com:

$$
A \begin{bmatrix} x_1 & x_2 \end{bmatrix}\begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}
$$

e, portanto:
$$
A = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}
$$

Por exemplo, se quiséssemos uma matriz cujos auto-valores são $0.7$ e $0.2$ e os auto-vetores correspondentes são $[1,0]^T$ e $[0.5, 0.5]^T$, então deveríamos calcular:

$$
A = \begin{bmatrix} 1 & 0.5 \\ 0 & 0.5 \end{bmatrix} \begin{bmatrix} 0.7 & 0 \\ 0  & 0.2 \end{bmatrix} \begin{bmatrix} 1 & 0.5 \\ 0 & 0.5 \end{bmatrix}^{-1}
$$

Encontre a matriz com autovetores $[1,0]^T$ e $[0.5, 0.5]^T$ e autovalores $0.7$ e $0.2$.

# Exercício 5
*Objetivo: relacionar autovalores ao colapso, explosão e ao equilíbrio de sistemas dinâmicos*

Quando aplicamos uma matriz sobre um vetor qualquer $v_0$, isto é, quando damos um passo em nosso sistema dinâmico, temos a situação:

$$
v_1 = A v_{0}
$$

Podemos aplicar novamente a matriz sobre $v_1$, encontrando:
$$
v_2 = A v_1 = A A v_0 = A^2 v_0.
$$

E, ao longo de $N$ iterações, teremos:
$$
v_N = A^N v_0
$$
---

Isso é  o que fizemos diversas vezes até este momento. Vamos agora re-escrever essa mesma equação na usando a forma de autovalores e autovetores. As passagens que seguem parecem um pouco longas porque as expressões são longas, mas são somente a consequência de trocar $A$ nas equações acima pela decomposição matricial:

$$
A = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}
$$

Aplicando essa mudança na primeira expressão, temos:

$$
v_1 = A v_0 = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} v_{0}
$$

Na segunda iteração, temos:

$$
v_2 = A v_1 = \left( A \right) A v_0 = \left( \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} \right) \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}v_0
$$

Veja que podemos simplificar a multiplicação $ \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} \begin{bmatrix} x_1 & x_2 \end{bmatrix}$ no meio da cadeia de multiplicações, já que ela é igual à identidade. Ficamos então com:

$$
v_2 = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}v_0
$$

Podemos resumir a multiplicação matricial $\begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix}$ como $\begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix}^2$, ficando com:

$$
v_2 = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} ^2 \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}v_0
$$

Fazendo uma nova multiplicação por $A$, vamos encontrar:
$$
v_3 = \left( A \right) v_2 = \left( \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} \right) \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} ^2 \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}v_0
$$

Novamente, podemos trocar a multiplicação matricial $ \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} \begin{bmatrix} x_1 & x_2 \end{bmatrix}$ pela identidade, e agrupar $\begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix}^2$ como $\begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix}^3$, ficando com:

$$
v_3 = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} ^3 \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} v_0
$$

Podemos fazer essa mesma operação $N$ vezes, ficando com:

$$
v_N = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} ^N \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} v_0
$$

Como $\begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} ^N = \begin{bmatrix} \lambda_1 ^N & 0 \\ 0 & \lambda_2 ^N \end{bmatrix}$, a expressão fica:

$$
v_N = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1^N & 0 \\ 0 & \lambda_2^N \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} v_0
$$

---

Veja que agora sabemos que:
$$
A^N = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1^N & 0 \\ 0 & \lambda_2^N \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}
$$ 

Usando valores à sua escolha e supondo que $A \in \mathbb{R}^{2\times 2}$, monte exemplos que mostrem o que acontece com os valores de $A^N$ se:

1. Todos os auto-valores forem positivos e menores que 1?
2. Somente um auto-valor for igual a 1, e os demais forem positivos e iguais a 1?
3. Um dos auto-valores de for maior que 1
4. Todos os auto-valores forem iguais a 1
5. Relacione as suas respostas anteriores aos conceitos de *explosão* (tender ao infinito ao longo de muitas iterações), *colapso* (tender a zero ao longo de muitas iterações) e *equilíbrio* (tender a um valor constante não-zero ao longo de muitas iteraçõees).
6. O algoritmo PageRank usa uma matriz para representar um sistema dinâmico. Esse sistema explode, colapsa ou entra em equilíbrio?



# Exercício 6
*Objetivo: relacionar colapso, explosão e equilíbrio a uma situação prática*

(a) Usando auto-valores e auto-vetores no caso das bicicletas de Montreal, mostre um argumento para que, independente das condições iniciais, a quantidade de bicicletas em cada estação sempre tenderá a uma mesma proporção.


In [ ]:
A = np.array( [ [0, 0.95, 0.05], [0.9, 0, 0.1], [0.7, 0.3, 0]]).T


(b) Usando auto-valores e auto-vetores no caso da população de carcarás, justifique o fato de que muito provavelmente a população deve tender a um equilíbrio ao longo de muitos meses.


In [ ]:
A = np.array([[0.8, 0.2], [-0.1,  1.1]])

(c) Partindo do caso da população de carcarás, suponha a seguinte situação. A população local decidiu que existem muitos carcarás nas redondezas, e por isso autorizou a caça. Com isso, ao fim do mês, ao invés de morrerem 20% dos carcarás existentes, morrem 80% dos carcarás existentes. Use auto-valores e auto-vetores para prever se esse processo de caça será eficaz para conter a população de carcarás. Após, confirme seu resultado usando uma simulação.

In [227]:
A = np.array([[0.2, 0.2], [-0.1,  1.1]]) # Depois da caça, a nova matriz A ficou assim!!!

(d) Após a publicação de imagens chocantes sobre a caça dos carcarás, foram aprovadas leis de proteção e cuidado ambiental que fazem com que a mortalidade dos carcarás de um mês para o outro seja de apenas 19%. Usando auto-valores e auto-vetores, determine qual é o efeito dessa lei, a longo prazo, para as populações de carcarás e sapos na região.

In [235]:
A = np.array([[0.81, 0.2], [-0.1,  1.1]]) # Depois da nova legislação, a nova matriz A ficou assim!!!

# Usar auto-valores e auto-vetores para tunar o comportamento de bots em um jogo

# Parte 2

# Usar uma matriz de covariância para representar dados (duas colunas)

# Relacionar a direção dos autovetores da matriz de covariância com a direção dos dados

# Verificar que autovetores da matriz de covariância são ortogonais entre si

# Usar os autovetores da matriz de covariância para fazer PCA e visualizar scatterplot de dados de alta dimensão

# Parte 3

# Analisar o erro de reconstrução ao usar PCA

# Construir PCA usando método de gradiente (manualmente)

# Construir PCA usando método de gradiente (autograd)

# Usar PCA do sklearn

# Parte 4

# Verificar o teorema SVD

[Demonstração do teorema](https://gregorygundersen.com/blog/2018/12/20/svd-proof/)

$ A = USV^T $
onde
$ U^T U = I$
$ V^T V = I$

* Colunas de $V$ são os autovetores de $A A^T$
* Autovetores de $U$ são autovetores de $A^T A$ 
* $S$ é uma matriz diagonal com 

# Manipular a matriz S em uma imagem e verificar os resultados

# Remover ruídos de uma imagem manipulando a matriz S

# Comprimir uma imagem modificando a matriz S

Projeto

Sistema de recomendação (desafio NetFlix)

# 